In [2]:
import os
# from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
import time
from google.cloud import bigquery
import requests
import numpy as np

# Querying the database


In [9]:
def query_gbq(start_date, end_date):
    query = """
        SELECT *
        FROM `wagon-bootcamp-802.my_dataset.new_table`
        WHERE created_at < {} AND created_at > {}
        ORDER BY created_at DESC
        LIMIT 100
    """
    # Run a Standard SQL query with the project set explicitly
    final_query = query.format(start_date, end_date)
    project_id = 'wagon-bootcamp-802'
    df = pd.read_gbq(final_query, project_id=project_id, dialect='standard')
    return df

In [10]:
start_date = '"2022-04-05 15:25:49+00:00"'
end_date = '"2022-04-05 15:24:59+00:00"'
df = query_gbq(start_date, end_date)

In [11]:
df.shape

(3, 4)

# Cleaning the data

## Removing Retweet indicator and @ mentions

In [4]:
df_clean = df.copy()
df_clean['text'] = df_clean['text'].str.replace(r'RT @\S* ', '')
df_clean['text'] = df_clean['text'].str.replace(r'@\S* ', '')
df_clean['text'] = df_clean['text'].str.replace(r'http\S*', '')

/tmp/ipykernel_922/4039097212.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean['text'] = df_clean['text'].str.replace(r'RT @\S* ', '')
/tmp/ipykernel_922/4039097212.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean['text'] = df_clean['text'].str.replace(r'@\S* ', '')
/tmp/ipykernel_922/4039097212.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean['text'] = df_clean['text'].str.replace(r'http\S*', '')


## Removing punct

In [5]:
import string
def punct_remove(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '') 
    return text
df_clean['text'] = df_clean['text'].apply(punct_remove)

## All lower

In [6]:
df_clean['text'] = df_clean['text'].apply(lambda row : row.lower())

## Removing numbers

In [7]:
def num_remove(text):
    text_rwkd = ''
    for car in text:
        text_rwkd += car if not car.isdigit() else ''
    return text_rwkd

df_clean['text'] = df_clean['text'].apply(num_remove)

## Removing stopwords

In [8]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def stop_remove(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    return " ".join([w for w in word_tokens if not w in stop_words])

df_clean['text'] = df_clean['text'].apply(stop_remove)
df_clean['text'] = df_clean['text'].str.replace(r' f ', ' f1 ')

In [9]:
df.head()

,created_at,text,id,lang
0,2022-04-04 17:03:48+00:00,RT @AstonMartinF1: Guess who's back? Let's do ...,1511026762075062279,en
1,2022-04-04 17:03:48+00:00,RT @AstonMartinF1: Guess who's back? Let's do ...,1511026761689186305,en
2,2022-04-04 17:03:46+00:00,RT @AstonMartinF1: Guess who's back? Let's do ...,1511026752759296003,en
3,2022-04-04 17:03:45+00:00,RT @AstonMartinF1: Guess who's back? Let's do ...,1511026751127932941,en
4,2022-04-04 17:03:44+00:00,RT @AstonMartinF1: Guess who's back? Let's do ...,1511026746530811913,en


In [10]:
df_clean.head()

,created_at,text,id,lang
0,2022-04-04 17:03:48+00:00,guess whos back lets seb 👊 f1 ausgp welcomebac...,1511026762075062279,en
1,2022-04-04 17:03:48+00:00,guess whos back lets seb 👊 f1 ausgp welcomebac...,1511026761689186305,en
2,2022-04-04 17:03:46+00:00,guess whos back lets seb 👊 f1 ausgp welcomebac...,1511026752759296003,en
3,2022-04-04 17:03:45+00:00,guess whos back lets seb 👊 f1 ausgp welcomebac...,1511026751127932941,en
4,2022-04-04 17:03:44+00:00,guess whos back lets seb 👊 f1 ausgp welcomebac...,1511026746530811913,en


In [11]:
drivers = {'Hamilton' : ['hamilton','lewis'],
           'Russel' : ['russel','georges'],
           'Perez' : ['perez','checo','sergio'],
           'Verstappen' : ['verstappen','max'],
           'Sainz' : ['sainz','carlos'],
           'Leclerc' : ['leclerc','charles'],
           'Ricciardo' : ['ricciardo', 'daniel'],
           'Norris' : ['norris','lando'],
           'Alonso' : ['alonso','fernando'],
           'Ocon' : ['ocon','esteban'],
           'Schumacher' : ['schumacher','mick'],
           'Magnussen' : ['magnussen','kevin'],
           'Bottas' : ['bottas','valtteri'],
           'Zhou' : ['zhou','guanyu'],
           'Gasly' : ['gasly','pierre'],
           'Tsunoda' : ['tsunoda','yuki'],
           'Stroll' : ['stroll','lance'],
           'Hulkenberg' : ['hulkenberg','nico'],
           'Albon' : ['albon','alexander'],
           'Latifi' : ['latifi','nicholas']}

In [12]:
nb_tweets = {}
for driver, names in drivers.items():
    mask = df_clean['text'].str.contains(f"{names[0]}", na=False)
    if len(names) > 1:
        for name in names[1:]:
            mask = mask | df_clean['text'].str.contains(f"{name}", na=False)
    nb_tweets[driver] = len(df_clean[mask])

In [13]:
nb_tweets

{'Hamilton': 2317,
 'Russel': 167,
 'Perez': 2143,
 'Verstappen': 2410,
 'Sainz': 431,
 'Leclerc': 1203,
 'Ricciardo': 2262,
 'Norris': 330,
 'Alonso': 453,
 'Ocon': 135,
 'Schumacher': 1177,
 'Magnussen': 105,
 'Bottas': 147,
 'Zhou': 897,
 'Gasly': 634,
 'Tsunoda': 240,
 'Stroll': 115,
 'Hulkenberg': 149,
 'Albon': 46,
 'Latifi': 82}

In [14]:
one = df_clean['text'].str.contains(f"gasly", na=False)
df_clean[one].iloc[4]['text']

'alphatauris transitions 🔥\u2060 \u2060 🎥 alphataurif \u2060 f1 pierregasly yukitsunoda motorsport'